In [3]:
import ee
ee.Authenticate()

True

In [ ]:
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langchain_groq import ChatGroq
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from dotenv import load_dotenv
from sarvamai import SarvamAI
from gtts import gTTS
import pygame
import re

load_dotenv()

# Initialize chat history
chat_history = ChatMessageHistory()

# Initialize the LLM for the chat model
llm = ChatGroq(model="Llama-3.3-70b-Versatile", api_key=os.getenv("GROQ_API_KEY"))

# Initialize SarvamAI client
sarvam_client = SarvamAI(api_subscription_key=os.getenv("SARVAM_API_KEY"))

# Language mapping for SarvamAI
SUPPORTED_LANGUAGES = {
    '1': ('English', 'en-IN'),
    '2': ('Hindi', 'hi-IN'),
    '3': ('Tamil', 'ta-IN'),
    '4': ('Telugu', 'te-IN'),
    '5': ('Kannada', 'kn-IN'),
    '6': ('Malayalam', 'ml-IN'),
    '7': ('Marathi', 'mr-IN'),
    '8': ('Gujarati', 'gu-IN'),
    '9': ('Punjabi', 'pa-IN'),
    '10': ('Bengali', 'bn-IN'),
    '11': ('Odia', 'od-IN'),
    '12': ('Assamese', 'as-IN'),
    '13': ('Urdu', 'ur-IN')
}

# Enhanced prompt template
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        """You are an advanced mineral excavation site analysis assistant that provides comprehensive insights about mining locations using real-time geological, economic, and market data based solely on latitude and longitude coordinates.

        CORE FUNCTIONALITY:
        - Accept ONLY latitude and longitude coordinates as input
        - Automatically identify location name, region, and geological context
        - Analyze mineral potential using satellite imagery and geological databases
        - Provide detailed economic and investment analysis
        - Generate actionable recommendations for mining industries
        - Respond in clear, structured format suitable for translation

        ANALYSIS REQUIREMENTS:
        Focus STRICTLY on the coordinates provided. You must autonomously gather and analyze all relevant information without requiring additional input data from the user.

        RESPONSE STRUCTURE:
        
        ### 🗺️ Mineral Excavation Site Analysis for {latitude}°N, {longitude}°E
        
        #### 1. 📍 Site Location and Geological Formation
        - **Coordinates**: {latitude}°N, {longitude}°E
        - **Identified Location**: [Auto-determine region/district name]
        - **Geological Formation**: Rock types, age, and mineral-bearing structures
        - **Ore Bodies**: Primary and secondary mineral deposits
        - **Terrain Analysis**: Topography, accessibility, and mining feasibility
        - **Infrastructure Assessment**: Roads, power, water, and transportation access
        
        #### 2. ⛏️ Primary Mineral Deposits Identified
        - **Dominant Minerals**: Specific minerals detected at coordinates
        - **Grade Analysis**: Ore grade percentages and quality specifications
        - **Reserve Estimates**: Proven, probable, and possible reserves (tons)
        - **Production Capacity**: Estimated annual extraction potential
        - **Extraction Methods**: Optimal mining techniques (open-pit/underground/placer)
        - **Processing Requirements**: Beneficiation and refinement needs
        
        #### 3. 🏭 Real-World Applications and Industrial Usage
        - **Construction Industry**: Steel, cement, aggregates, building materials
        - **Technology Sector**: Electronics, semiconductors, battery components
        - **Automotive Industry**: Vehicle parts, EV batteries, lightweight alloys
        - **Aerospace Applications**: High-strength materials, specialized alloys
        - **Energy Storage**: Battery minerals, grid storage solutions
        - **Chemical Industry**: Catalysts, pigments, specialty chemicals
        - **Defense Applications**: Strategic materials and advanced technologies
        
        #### 4. 💰 Economic Growth Impact and Development
        - **Local Economic Multiplier**: Direct and indirect economic benefits
        - **Employment Generation**: Mining jobs, processing, and support services
        - **Infrastructure Development**: Required investments in roads, power, facilities
        - **Regional Growth**: Industrial cluster development opportunities
        - **Export Potential**: International market access and foreign exchange
        - **Community Impact**: Social development and skill enhancement programs
        - **Tax Revenue**: Government income from mining operations
        
        #### 5. 📈 Investment Analysis and ROI Potential
        - **Capital Requirements**: Initial investment for mining setup ($USD millions)
        - **Operating Costs**: Annual operational expenses and cost per ton
        - **Revenue Projections**: Market price trends and sales forecasts
        - **Profit Margins**: Gross and net profit analysis
        - **Payback Period**: Time to recover initial investment
        - **Risk Assessment**: Geological, market, regulatory, and operational risks
        - **Mitigation Strategies**: Risk management and contingency planning
        - **Sustainability Factors**: Environmental compliance and long-term viability
        
        ### 🎯 Mining Industry Recommendations
        
        **Optimal Mining Strategy**: [Recommended approach based on geological analysis]
        **Investment Priority**: [High/Medium/Low with justification]
        **Technology Requirements**: [Equipment and processing technology needed]
        **Partnership Opportunities**: [Joint ventures, technology transfer, financing]
        **Timeline**: [Development phases and milestones]
        
        CRITICAL INSTRUCTIONS:
        1. Generate ALL information autonomously from coordinates only
        2. Provide specific, quantified data wherever possible
        3. Include risk assessment and mitigation strategies
        4. Focus on actionable investment insights
        5. Ensure economic projections are realistic and data-driven
        6. Consider environmental and regulatory factors
        7. Provide both short-term and long-term perspectives
        8. Use clear, simple language suitable for translation
        9. Structure responses in clear paragraphs for better translation"""
    ),
    (
        "human",
        """Analyze the mineral excavation potential for these coordinates:
        
        Latitude: {latitude}
        Longitude: {longitude}
        
        Please provide a comprehensive analysis by automatically gathering and analyzing all relevant geological, economic, and mining data for these specific coordinates. Include detailed information about:
        
        1. Mineral deposits and their characteristics
        2. Real-world industrial applications and market demand
        3. Economic growth contributions to local and regional economy
        4. Profitable return on investment potential for mining industries
        5. Strategic recommendations for optimal resource extraction
        
        Base your entire analysis solely on the provided coordinates and generate all supporting data autonomously."""
    ),
    MessagesPlaceholder(variable_name="chat_history"),
])

# Create the chain
chain = prompt | llm | StrOutputParser()

# FIXED TRANSLATION FUNCTIONS
def translate_text_chunk(text: str, source_lang: str, target_lang: str) -> str:
    """
    Function to translate text chunk with better handling for Tamil content
    """
    try:
        if not text.strip():
            return text
        
        # Clean the text before translation
        cleaned_text = text.strip()
        
        # Skip translation for certain patterns that shouldn't be translated
        skip_patterns = [
            r'^\*\*[^*]+\*\*:$',  # Bold headers like **Coordinates**:
            r'^\d+\.\d+°[NS], \d+\.\d+°[EW]$',  # Coordinates
            r'^\$[\d,]+',  # Dollar amounts
            r'^\d+%',  # Percentages
            r'^\d+\s*(tons?|million|billion)',  # Quantities
        ]
        
        for pattern in skip_patterns:
            if re.match(pattern, cleaned_text):
                return text
        
        response = sarvam_client.text.translate(
            input=cleaned_text,
            source_language_code=source_lang,
            target_language_code=target_lang
        )
        return response.translated_text
    except Exception as e:
        print(f"Translation error for text '{text[:50]}...': {str(e)}")
        return text

def text_to_speech(text: str, lang: str, output_file: str = "output.mp3") -> str:
    """
    Function to convert text to speech and save as audio file.
    """
    try:
        # Map SarvamAI language codes to gTTS codes
        lang_mapping = {
            'hi-IN': 'hi', 'ta-IN': 'ta', 'te-IN': 'te', 'kn-IN': 'kn',
            'ml-IN': 'ml', 'mr-IN': 'mr', 'gu-IN': 'gu', 'pa-IN': 'pa',
            'bn-IN': 'bn', 'en-IN': 'en'
        }
        
        tts_lang = lang_mapping.get(lang, 'en')
        tts = gTTS(text=text, lang=tts_lang)
        tts.save(output_file)
        return output_file
    except Exception as e:
        return None

def play_audio_optional(audio_file: str):
    """
    Play audio file using pygame with user control
    """
    try:
        pygame.mixer.init()
        pygame.mixer.music.load(audio_file)
        pygame.mixer.music.play()
        
        print("🔊 Audio is playing... Press 's' to stop, 'p' to pause/resume, or wait for completion:")
        
        while pygame.mixer.music.get_busy():
            # Check for user input without blocking
            import select
            import sys
            
            if select.select([sys.stdin], [], [], 0.1)[0]:
                user_input = sys.stdin.readline().strip().lower()
                if user_input == 's':
                    pygame.mixer.music.stop()
                    print("🛑 Audio stopped by user")
                    break
                elif user_input == 'p':
                    if pygame.mixer.music.get_busy():
                        pygame.mixer.music.pause()
                        print("⏸️ Audio paused")
                    else:
                        pygame.mixer.music.unpause()
                        print("▶️ Audio resumed")
            
            pygame.time.wait(100)
            
    except Exception as e:
        print(f"Audio playback error: {str(e)}")

def print_header(title: str, width: int = 80):
    """Print a neat header"""
    print("\n" + "=" * width)
    print(f"{title:^{width}}")
    print("=" * width)

def print_section(title: str, width: int = 60):
    """Print a neat section header"""
    print(f"\n{title}")
    print("-" * width)

# FIXED STREAMING TRANSLATION FUNCTIONALITY
def neat_multilingual_stream_analysis(latitude: float, longitude: float, target_language: str):
    """
    Stream the mineral excavation analysis response with FIXED translation for all content
    """
    try:
        # Prepare the input for the chain
        input_data = {
            "latitude": latitude,
            "longitude": longitude,
            "chat_history": chat_history.messages
        }
        
        print_header(f"🗺️ MINERAL EXCAVATION SITE ANALYSIS")
        print(f"📍 Coordinates: {latitude}°N, {longitude}°E")
        print(f"🌐 Output Language: {target_language}")
        
        # Initialize variables for neat streaming
        english_content = []
        
        print_section("📊 ENGLISH ANALYSIS", 60)
        
        # Stream English content with neat formatting
        for chunk in chain.stream(input_data):
            print(chunk, end="", flush=True)
            english_content.append(chunk)
        
        # Get complete English response
        complete_english_response = "".join(english_content)
        
        # Translate if target language is not English
        if target_language != 'en-IN':
            print_section(f"🌐 TRANSLATED VERSION ({target_language})", 60)
            
            try:
                # Split content into smaller, more manageable chunks for translation
                lines = complete_english_response.split('\n')
                translated_lines = []
                
                for line in lines:
                    if line.strip():
                        # Check if it's a header line
                        if line.startswith('#'):
                            # Translate header
                            translated_line = translate_text_chunk(line.strip(), 'en-IN', target_language)
                            print(f"{translated_line}")
                            translated_lines.append(translated_line)
                        elif line.startswith('-') and '**' in line:
                            # Handle bullet points with bold text
                            parts = line.split(':', 1)
                            if len(parts) == 2:
                                label_part = parts[0].strip()
                                content_part = parts[1].strip()
                                
                                # Translate both parts separately
                                translated_label = translate_text_chunk(label_part, 'en-IN', target_language)
                                translated_content = translate_text_chunk(content_part, 'en-IN', target_language)
                                
                                translated_line = f"{translated_label}: {translated_content}"
                                print(f"{translated_line}")
                                translated_lines.append(translated_line)
                            else:
                                # Translate the whole line
                                translated_line = translate_text_chunk(line.strip(), 'en-IN', target_language)
                                print(f"{translated_line}")
                                translated_lines.append(translated_line)
                        elif line.strip().startswith('**') and line.strip().endswith('**'):
                            # Handle standalone bold text
                            translated_line = translate_text_chunk(line.strip(), 'en-IN', target_language)
                            print(f"{translated_line}")
                            translated_lines.append(translated_line)
                        else:
                            # Regular content line
                            if len(line.strip()) > 10:  # Only translate substantial content
                                translated_line = translate_text_chunk(line.strip(), 'en-IN', target_language)
                                print(f"{translated_line}")
                                translated_lines.append(translated_line)
                            else:
                                print(line)
                                translated_lines.append(line)
                    else:
                        # Empty line
                        print()
                        translated_lines.append("")
                
                complete_translated_response = "\n".join(translated_lines)
                
                # Store both versions in chat history
                chat_history.add_user_message(f"Analyze coordinates: {latitude}, {longitude} (Language: {target_language})")
                chat_history.add_ai_message(f"English: {complete_english_response}\n\nTranslated: {complete_translated_response}")
                
                return complete_english_response, complete_translated_response
                
            except Exception as e:
                print(f"\n❌ Translation Error: {str(e)}")
                chat_history.add_user_message(f"Analyze coordinates: {latitude}, {longitude}")
                chat_history.add_ai_message(complete_english_response)
                return complete_english_response, None
        else:
            # Store English response in chat history
            chat_history.add_user_message(f"Analyze coordinates: {latitude}, {longitude}")
            chat_history.add_ai_message(complete_english_response)
            return complete_english_response, None
        
    except Exception as e:
        error_message = f"Error during analysis: {str(e)}"
        print(f"\n❌ {error_message}")
        return error_message, None

# MAIN CONSOLE APPLICATION
def main_console_app():
    """
    Main console application with fixed multilingual support
    """
    print_header("🗺️ MULTILINGUAL MINERAL EXCAVATION SITE ANALYZER")
    print("🌐 Supports 13 Indian languages with FIXED content translation")
    print("⌨️ Text-based input for coordinates")
    print("🔄 Live streaming translation with neat formatting")
    print("🔊 Optional audio playback (not forced)")
    
    while True:
        try:
            print_section("📍 COORDINATE INPUT")
            print("Enter coordinates manually or type 'exit' to quit")
            
            # Manual input only
            lat_input = input("➤ Enter Latitude: ").strip()
            if lat_input.lower() == 'exit':
                print_header("👋 GOODBYE!")
                break
                
            lon_input = input("➤ Enter Longitude: ").strip()
            if lon_input.lower() == 'exit':
                print_header("👋 GOODBYE!")
                break
            
            try:
                lat = float(lat_input)
                lon = float(lon_input)
            except ValueError:
                print("❌ Invalid coordinates. Please enter valid numeric values.")
                continue
            
            # Language selection with neat formatting
            print_section("🌐 LANGUAGE SELECTION")
            for key, (name, code) in SUPPORTED_LANGUAGES.items():
                print(f"{key:2}. {name:12} ({code})")
            
            lang_choice = input("\n➤ Select language (1-13): ").strip()
            if lang_choice not in SUPPORTED_LANGUAGES:
                print("❌ Invalid language choice. Using English.")
                target_lang_name, target_lang_code = 'English', 'en-IN'
            else:
                target_lang_name, target_lang_code = SUPPORTED_LANGUAGES[lang_choice]
            
            print(f"✅ Selected language: {target_lang_name} ({target_lang_code})")
            
            # Perform neat analysis with FIXED translation
            english_response, translated_response = neat_multilingual_stream_analysis(
                lat, lon, target_lang_code
            )
            
            print_header("✅ ANALYSIS COMPLETED SUCCESSFULLY!")
            
            # OPTIONAL AUDIO SECTION
            if translated_response and target_lang_code != 'en-IN':
                print_section("🔊 OPTIONAL AUDIO FEATURES")
                print("Would you like to:")
                print("1. Generate and play translated audio")
                print("2. Generate audio file only (no playback)")
                print("3. Skip audio completely")
                
                audio_choice = input("\n➤ Select audio option (1-3): ").strip()
                
                if audio_choice == '1':
                    try:
                        print("🔊 Generating translated audio...")
                        audio_file = text_to_speech(
                            translated_response, 
                            target_lang_code, 
                            f"analysis_{target_lang_code}.mp3"
                        )
                        if audio_file:
                            print("🔊 Audio generated successfully!")
                            play_audio_optional(audio_file)
                    except Exception as e:
                        print(f"❌ Audio generation failed: {str(e)}")
                        
                elif audio_choice == '2':
                    try:
                        print("🔊 Generating audio file...")
                        audio_file = text_to_speech(
                            translated_response, 
                            target_lang_code, 
                            f"analysis_{target_lang_code}.mp3"
                        )
                        if audio_file:
                            print(f"✅ Audio file saved as: {audio_file}")
                            print("You can play it later using any audio player.")
                    except Exception as e:
                        print(f"❌ Audio generation failed: {str(e)}")
                        
                elif audio_choice == '3':
                    print("✅ Skipping audio generation.")
                    
            elif target_lang_code == 'en-IN':
                # Optional English audio
                audio_choice = input("\n🔊 Generate English audio? (y/n): ").lower().strip()
                if audio_choice == 'y':
                    try:
                        print("🔊 Generating English audio...")
                        audio_file = text_to_speech(
                            english_response, 
                            'en-IN', 
                            f"analysis_english.mp3"
                        )
                        if audio_file:
                            print("🔊 Audio generated successfully!")
                            play_choice = input("Play now? (y/n): ").lower().strip()
                            if play_choice == 'y':
                                play_audio_optional(audio_file)
                            else:
                                print(f"✅ Audio file saved as: {audio_file}")
                    except Exception as e:
                        print(f"❌ Audio generation failed: {str(e)}")
            
            # Ask if user wants to continue
            continue_analysis = input("\n🔄 Analyze another location? (y/n): ").lower().strip()
            if continue_analysis != 'y':
                break
                
        except KeyboardInterrupt:
            print_header("👋 GOODBYE!")
            break
        except Exception as e:
            print(f"❌ Error: {str(e)}")

# NEAT CHAT HISTORY VIEWER
def view_chat_history():
    """
    Display chat history with neat formatting
    """
    if not chat_history.messages:
        print("📭 No analysis history available.")
        return
    
    print_header("💬 MULTILINGUAL ANALYSIS HISTORY")
    
    for i, message in enumerate(chat_history.messages):
        if isinstance(message, HumanMessage):
            print_section(f"🔍 Query {i//2 + 1}")
            print(f"📍 {message.content}")
        elif isinstance(message, AIMessage):
            print_section(f"📊 Analysis Results {i//2 + 1}")
            
            # Check if message contains translation
            if "Translated:" in message.content:
                english_part, translated_part = message.content.split("Translated:", 1)
                
                print("\n🇺🇸 **English Version:**")
                print(english_part.replace("English:", "").strip()[:500] + "...")
                
                print("\n🌐 **Translated Version:**")
                print(translated_part.strip()[:500] + "...")
            else:
                print(message.content[:500] + "...")

# NEAT LANGUAGE SUPPORT DISPLAY
def show_language_support():
    """
    Display supported languages and features with neat formatting
    """
    print_header("🌐 SUPPORTED LANGUAGES & FEATURES")
    
    print_section("📋 AVAILABLE LANGUAGES")
    for key, (name, code) in SUPPORTED_LANGUAGES.items():
        print(f"{key:2}. {name:15} ({code})")
    
    print_section("🔧 ENHANCED FEATURES")
    features = [
        "🔄 Real-time Translation: Content translated as it's generated",
        "🔤 FIXED Content Translation: All content properly translated",
        "🔊 Optional Audio: User-controlled audio generation and playback",
        "⌨️ Text Input: Manual coordinate entry",
        "💬 Chat History: Maintains conversation in multiple languages",
        "🎯 Live Processing: See both English and translated content",
        "📊 Neat Formatting: Clean, organized output display",
        "⏯️ Audio Controls: Play, pause, stop audio during playback"
    ]
    
    for feature in features:
        print(f"  • {feature}")

# MAIN EXECUTION
if __name__ == "__main__":
    while True:
        print_header("🗺️ MULTILINGUAL MINERAL EXCAVATION SITE ANALYZER")
        print("🔄 WITH FIXED CONTENT TRANSLATION")
        print("⌨️ TEXT-BASED INPUT ONLY")
        
        print_section("📋 MAIN MENU")
        print("1. Start Analysis")
        print("2. View Chat History")
        print("3. Language Support Info")
        print("4. Exit")
        
        choice = input("\n➤ Select option (1-4): ").strip()
        
        if choice == '1':
            main_console_app()
        elif choice == '2':
            view_chat_history()
        elif choice == '3':
            show_language_support()
        elif choice == '4':
            print_header("👋 THANK YOU FOR USING THE ANALYZER!")
            break
        else:
            print("❌ Invalid choice. Please select 1-4.")
            

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html

                🗺️ MULTILINGUAL MINERAL EXCAVATION SITE ANALYZER                
🔄 WITH FIXED CONTENT TRANSLATION
⌨️ TEXT-BASED INPUT ONLY

📋 MAIN MENU
------------------------------------------------------------
1. Start Analysis
2. View Chat History
3. Language Support Info
4. Exit



➤ Select option (1-4):  3



                        🌐 SUPPORTED LANGUAGES & FEATURES                        

📋 AVAILABLE LANGUAGES
------------------------------------------------------------
1 . English         (en-IN)
2 . Hindi           (hi-IN)
3 . Tamil           (ta-IN)
4 . Telugu          (te-IN)
5 . Kannada         (kn-IN)
6 . Malayalam       (ml-IN)
7 . Marathi         (mr-IN)
8 . Gujarati        (gu-IN)
9 . Punjabi         (pa-IN)
10. Bengali         (bn-IN)
11. Odia            (od-IN)
12. Assamese        (as-IN)
13. Urdu            (ur-IN)

🔧 ENHANCED FEATURES
------------------------------------------------------------
  • 🔄 Real-time Translation: Content translated as it's generated
  • 🔤 FIXED Content Translation: All content properly translated
  • 🔊 Optional Audio: User-controlled audio generation and playback
  • ⌨️ Text Input: Manual coordinate entry
  • 💬 Chat History: Maintains conversation in multiple languages
  • 🎯 Live Processing: See both English and translated content
  • 📊 Neat Formatt


➤ Select option (1-4):  1



                🗺️ MULTILINGUAL MINERAL EXCAVATION SITE ANALYZER                
🌐 Supports 13 Indian languages with FIXED content translation
⌨️ Text-based input for coordinates
🔄 Live streaming translation with neat formatting
🔊 Optional audio playback (not forced)

📍 COORDINATE INPUT
------------------------------------------------------------
Enter coordinates manually or type 'exit' to quit


...


🌐 LANGUAGE SELECTION
------------------------------------------------------------
1 . English      (en-IN)
2 . Hindi        (hi-IN)
3 . Tamil        (ta-IN)
4 . Telugu       (te-IN)
5 . Kannada      (kn-IN)
6 . Malayalam    (ml-IN)
7 . Marathi      (mr-IN)
8 . Gujarati     (gu-IN)
9 . Punjabi      (pa-IN)
10. Bengali      (bn-IN)
11. Odia         (od-IN)
12. Assamese     (as-IN)
13. Urdu         (ur-IN)



➤ Select language (1-13):  3


✅ Selected language: Tamil (ta-IN)

                      🗺️ MINERAL EXCAVATION SITE ANALYSIS                       
📍 Coordinates: 16.2915189°N, 80.4541588°E
🌐 Output Language: ta-IN

📊 ENGLISH ANALYSIS
------------------------------------------------------------
### 🗺️ Mineral Excavation Site Analysis for 16.2915189°N, 80.4541588°E

#### 1. 📍 Site Location and Geological Formation
- **Coordinates**: 16.2915189°N, 80.4541588°E
- **Identified Location**: The coordinates point to a location in the state of Andhra Pradesh, India, specifically within the Guntur district.
- **Geological Formation**: The area is characterized by a mix of Precambrian and Phanerozoic rocks, with significant deposits of limestone, dolomite, and quartzite. These formations are indicative of a region with substantial mineral potential, including iron ore, copper, and limestone.
- **Ore Bodies**: Primary mineral deposits identified include iron ore (hematite and magnetite), copper (chalcopyrite and bornite), and 


➤ Select audio option (1-3):  2


🔊 Generating audio file...
✅ Audio file saved as: analysis_ta-IN.mp3
You can play it later using any audio player.
